Maybe we define new structure which captures the input parameters specific to each problem in one place (`scale_factor`).
We need to define a range on `d` that is also function dependent (that we could adjust by hand). 

We need to generate three graphs: 
- Histogram of Number of `loc_min` points were found, (so outputs of the Optim routine) and what percentage of them is within a small distance of a critical point of the approximant. As a function of the degree `d` of the approximant.


In [ ]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

In [ ]:
kwel = Globtim.GaussianParams([0.26089820418282555 0.7344607224948916; -0.03017402660215973 -0.08830141199871192; -0.5923178162494483 0.7056820121831168; 0.17219769077638444 0.23048326052302262; -0.24763641218779517 -0.7815397497735926; -0.4673401509337077 -0.20722331050611642; -0.7477436163328894 -0.2190129770253087; -0.001999373470356236 -0.2950834352880266; -0.3120301213620158 0.16354534796517442; 0.69272872993026 0.4029772296623259; -0.5931618012868126 -0.7334896493870542; -0.7295746099547044 0.01794672027273103], [0.3594863378910475, 0.26981260289694575, 0.5230637389101772, 0.1277917080242205, 0.11533967613820623, 0.38278982674149603, 0.45620512616814934, 0.1910983217345807, 0.2170627012236726, 0.4817586065231313, 0.0952629299826123, 0.5767317212090373], [-1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0])

rand_gaussian_closure = (x) -> rand_gaussian(x, kwel)
f = rand_gaussian_closure;
f([1.0, 1.0])

Can a function export "Optional attributes" like just some constants or its optimal domain of definition? --> works. 

In [ ]:
d_min, d_max = 3, 30
TD = 0.1
# gaussian parameters (previously from load_function_params)
TR = test_input(f;
    dim=2,
    center=[0.0, 0.0],
    GN=120,
    sample_range=1.0,
    tolerance=3e-4)

@polyvar(x[1:TR.dim]); # Define polynomial ring 

In [ ]:
results = analyze_degrees(TR, x, d_min, d_max, step=1, tol_dist=TD)

In [ ]:
fig_1 = plot_discrete_l2(new_results, d_min, d_max, 1)
# Commented out save command
# save("discrete_l2.pdf", fig_1)
display(fig_1)

In [ ]:
fig_2 = capture_histogram(new_results, d_min, d_max, 1, show_legend=false)
# Commented out save command
# save("histogram.pdf", fig_2)
display(fig_2)

In [ ]:
fig_3 = plot_convergence_analysis(results, d_min, d_max, 1, show_legend=false)
# Commented out save command
# save("convergence_analysis.pdf", fig_3)
display(fig_3)

In [ ]:
RT = results[28]
df_t = RT.df
df_m = RT.df_min;
inside_mask = points_in_hypercube(df_t, TR)
values_mask = points_in_range(df_t, TR, .8)
df_minimizers = df_t[values_mask.&inside_mask, :] # has both `x` (raw) and `y` (optimized)
pol_cheb = Constructor(TR, 8, basis=:chebyshev);


In [ ]:
fig_4 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_minimizers, df_m, show_captured=false)

In [ ]:
stats = analyze_converged_points(df_minimizers, TR, results, d_min, d_max, 1)
avg_distances = stats["avg_distances"]


In [ ]:
fig_5 = plot_distance_statistics(stats)
# Commented out save command
# save("distance_statistics.pdf", fig_5)